In [1]:
# Generate demand for simulation analysis

In [2]:
import itertools
import numpy as np
from scipy.stats import norm
from scipy.stats import poisson
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [3]:
# Consider two rate classes for simplicity
n_class = 2
# Consider Length-of-Stay (los) of upto three days
los = 3
capacity = 50
# Consider two demand scenario, by changing capacity
demand_intensity = np.array([0.9, 1.5])
# Slope for linear demand curve
slope = np.array([-0.1, -0.15])
# Nightly rates for Monday stay night through Sunday stay night
# Higher weekday rates imply business hotel settings and higher weekend rates imply resorts
init_rate1 = np.array([135, 135, 135, 135, 135, 108, 108])
init_rate2 = np.array([115, 115, 115, 115, 115, 92, 92])

In [4]:
# Take the slope of -0.1 for rate class 1, then the mean true demand point for any arrival date/lengthof-
# stay combination is passed through by the line defined by this slope; 30 points on the price 
# axis for each arrival date/length-of-stay combination over the first 21 simulation days 
# are randomly generated from the interval (init_rate-50, init_rate+50)
# Sunday-one night stay as an example
drawsize = 30
# Random draw for one night stay for each day of week
random_rates1_one = [np.random.randint(init_rate1[i]-50, init_rate1[i]+50+1, drawsize) 
                     for i in range(7)]
random_rates2_one = [np.random.randint(init_rate2[i]-50, init_rate2[i]+50+1, drawsize) 
                     for i in range(7)]

# Construct rates for more than one night stay, up to 3-night stay
# Sunday night arrival, two night stay rates equal sunday one night + monday one night stay
random_rates1_two = [random_rates1_one[i%7] 
                     + random_rates1_one[(i+1)%7] 
                     for i in range(7)]

random_rates2_two = [random_rates2_one[i%7] 
                     + random_rates2_one[(i+1)%7] 
                     for i in range(7)]

random_rates1_three = [random_rates1_one[i%7] 
                       + random_rates1_one[(i+1)%7] 
                       + random_rates1_one[(i+2)%7] 
                       for i in range(7)]

random_rates2_three = [random_rates2_one[i%7] 
                       + random_rates2_one[(i+1)%7] 
                       + random_rates2_one[(i+2)%7] 
                       for i in range(7)]

In [5]:
# Change type to numpy array
random_rates1_one = np.array(random_rates1_one).reshape(7, drawsize)
random_rates1_two = np.array(random_rates1_two).reshape(7, drawsize)
random_rates1_three = np.array(random_rates1_three).reshape(7, drawsize)
randomRates1 = [(random_rates1_one[i], random_rates1_two[i], random_rates1_three[i]) 
                 for i in range(7)]

In [6]:
# Reshape array, where axis 0 = day of week, axis 1 = length of stay, and axis 3 = random draw
randomRates1 = np.array(randomRates1).reshape(7, 3, drawsize)

In [7]:
randomRates1[0]

array([[171, 133, 135, 105, 118, 179, 183,  92, 156,  90, 173,  93, 185,
        174,  86, 177, 127, 120, 161, 157,  93, 149, 114, 163, 152, 157,
        140, 146, 177, 120],
       [274, 221, 226, 282, 279, 325, 362, 181, 277, 264, 352, 222, 314,
        302, 244, 308, 231, 263, 255, 271, 216, 286, 263, 330, 280, 277,
        229, 331, 284, 239],
       [454, 396, 329, 447, 390, 486, 536, 318, 398, 379, 456, 318, 425,
        469, 341, 467, 337, 430, 403, 384, 346, 377, 356, 499, 441, 448,
        379, 466, 449, 404]])

In [8]:
# Check accuracy, result should be similar to init_rate
np.mean(randomRates1, axis = 2)

array([[140.86666667, 272.93333333, 410.93333333],
       [132.06666667, 270.06666667, 394.76666667],
       [138.        , 262.7       , 395.8       ],
       [124.7       , 257.8       , 366.46666667],
       [133.1       , 241.76666667, 353.46666667],
       [108.66666667, 220.36666667, 361.23333333],
       [111.7       , 252.56666667, 384.63333333]])

In [9]:
# Construct rates for more than one night stay, up to 3-night stay
init_rate1

array([135, 135, 135, 135, 135, 108, 108])

In [10]:
# Calculate averate rates for each arrival day of week and los combination
rates1_arrival_los = [[init_rate1[i],
                      init_rate1[i] + init_rate1[(i+1)%7],
                      init_rate1[i] + init_rate1[(i+1)%7] + init_rate1[(i+2)%7]] 
                      for i in range(7)]
# Store it as a numpy array
rates1_arrival_los = np.array(rates1_arrival_los)

In [11]:
rates1_arrival_los

array([[135, 270, 405],
       [135, 270, 405],
       [135, 270, 405],
       [135, 270, 378],
       [135, 243, 351],
       [108, 216, 351],
       [108, 243, 378]])

In [12]:
# Calculate y-intercepts, assuming los distribution for a stay night is 1/3, 1/3, and 1/3
# There exist one 1-night stay, two 2-night stays and three 3-night stays that cover 
# stay night in question.
# Each rate class contribute half of the demand for each stay night
half_demand = 0.5 * capacity * demand_intensity

intercepts_rate1 = [[half_demand[0] * 1/3 - slope[0] * rates1_arrival_los[i, 0],
                   half_demand[0] * 1/3 * 1/2 - slope[0] * rates1_arrival_los[(i-1+7)%7, 1],
                   half_demand[0] * 1/3 * 1/3 - slope[0] * rates1_arrival_los[(i-2+7)%7, 2]] 
                    for i in range(7)]

intercepts_rate1 = np.array(intercepts_rate1)

In [13]:
slope[0], intercepts_rate1

(-0.1,
 array([[21.  , 28.05, 37.6 ],
        [21.  , 30.75, 40.3 ],
        [21.  , 30.75, 43.  ],
        [21.  , 30.75, 43.  ],
        [21.  , 30.75, 43.  ],
        [18.3 , 28.05, 40.3 ],
        [18.3 , 25.35, 37.6 ]]))

In [14]:
# Mean of Poisson demand that is generated by the slope and y-intercept
# One mean demand for each of the 30 randomly drawn price
meanDemand_rate1 = [intercepts_rate1[i,j] + slope[0] * randomRates1[i, j, k] 
                    for i, j, k in itertools.product(range(7), range(los), range(drawsize))]
meanDemand_rate1 = np.array(meanDemand_rate1).reshape(7, los, drawsize)
zeros = np.zeros(7 * los * drawsize).reshape(7, los, drawsize)
# Truncate mean demand at zero
meanDemand_rate1 = np.maximum(meanDemand_rate1, zeros)

In [15]:
# Mean demand rate for a Sunday arrival (including 1-, 2-, and 3-night stay) 
meanDemand_rate1[0]

array([[ 3.9 ,  7.7 ,  7.5 , 10.5 ,  9.2 ,  3.1 ,  2.7 , 11.8 ,  5.4 ,
        12.  ,  3.7 , 11.7 ,  2.5 ,  3.6 , 12.4 ,  3.3 ,  8.3 ,  9.  ,
         4.9 ,  5.3 , 11.7 ,  6.1 ,  9.6 ,  4.7 ,  5.8 ,  5.3 ,  7.  ,
         6.4 ,  3.3 ,  9.  ],
       [ 0.65,  5.95,  5.45,  0.  ,  0.15,  0.  ,  0.  ,  9.95,  0.35,
         1.65,  0.  ,  5.85,  0.  ,  0.  ,  3.65,  0.  ,  4.95,  1.75,
         2.55,  0.95,  6.45,  0.  ,  1.75,  0.  ,  0.05,  0.35,  5.15,
         0.  ,  0.  ,  4.15],
       [ 0.  ,  0.  ,  4.7 ,  0.  ,  0.  ,  0.  ,  0.  ,  5.8 ,  0.  ,
         0.  ,  0.  ,  5.8 ,  0.  ,  0.  ,  3.5 ,  0.  ,  3.9 ,  0.  ,
         0.  ,  0.  ,  3.  ,  0.  ,  2.  ,  0.  ,  0.  ,  0.  ,  0.  ,
         0.  ,  0.  ,  0.  ]])

In [16]:
# Check total mean demand for each day of week for each random drawn price
totalDemand_rate1 = [[np.sum(meanDemand_rate1[i,:,j]) 
                     + np.sum(meanDemand_rate1[(i-1+7)%7, 1:, j]) 
                     + np.sum(meanDemand_rate1[(i-2+7)%7, 2:, j]) 
                     for i in range(7)] for j in range(drawsize)]

In [17]:
# Check total mean demand for each day of week
totalDemand_rate1 = np.array(totalDemand_rate1).reshape(7, drawsize)
np.mean(totalDemand_rate1, axis=1)

array([18.82333333, 33.145     , 38.27333333, 22.90833333, 19.68333333,
       38.61      , 27.85666667])

In [18]:
# Generate random demand for each day of week and length of stay combination
randomDemand_rate1 = [poisson.rvs(mu, size=1) for mu in np.nditer(meanDemand_rate1)]
randomDemand_rate1 = np.array(randomDemand_rate1).reshape(7, los, drawsize)

In [19]:
randomDemand_rate1[0]

array([[ 6,  9,  6,  8, 18,  0,  2, 12,  6,  9,  3, 12,  2,  0, 13,  5,
        10, 13,  7,  5, 12,  7, 12,  1,  6,  7,  5,  4,  8,  7],
       [ 0,  4,  1,  0,  0,  0,  0,  5,  0,  1,  0,  5,  0,  0,  3,  0,
         8,  1,  4,  2,  1,  0,  1,  0,  0,  2,  5,  0,  0,  6],
       [ 0,  0,  5,  0,  0,  0,  0,  9,  0,  0,  0,  5,  0,  0,  3,  0,
         3,  0,  0,  0,  5,  0,  2,  0,  0,  0,  0,  0,  0,  0]])